## Garcilaso de la Vega: Power, Bodies, and the Poetic Gaze

Garcilaso de la Vega is one of the Spanish language's unavoidable reads. Despite his relatively short life—he died in 1536, at age 35—he was a prolific writer, and seminal in his time. He lived multiple lives in one, as a member of Charles V's court, an active member of the Spanish military, and of course a poet. The literature on Garcilaso's poetry tends to focus on two big features. Leah Middlebrook paints him in _Imperial Lyric: New Poetry and New Subjects in Modern Spain_ as a witness to a pivotal moment in Spanish history: the country's consolidation into a global empire, with all the tremendous consequences that transition entailed. Middlebrook presses the idea that Garcilaso was constantly thinking about “the imperial subject” and his identities and responsibilities in a changing world (Middlebrook, 96-100). Her interpretation intersects somewhat with Ignacio Navarrete's reading in _Orphans of Petrarch: Poetry and Theory in the Spanish Renaissance_. He argues Garcilaso's poetry reflects the author's consciousness of courtly manners, and how those changed in lockstep with Spain's transition to empire. Navarrete too reads Garcilaso as obsessed with the subject, but more so in relation to his multiple identities as courtier, poet, and soldier. Navarrete traces that obsession through Garcilaso's writing on bodies and gazes, which he interprets as a poetic retooling of the individual (Navarrete, “Boscán, Garcilaso, and the Codes of Erotic Poetry”).

I intend to investigate those claims in my project. I have a parallel interest too; I find Garcilaso's way of writing about nature intriguing, and I want to see if topic modeling will pick up on patterns in the language that might reveal a greater meaning to the few phrases that have caught my attention. I am speaking, for example, of Garcilaso's "Soneto XI," which begins with "hermosas ninfas, que, en el río metidas," which translates to "beautiful nymphs who, bathing in the river". Garcilaso almost seems to see nature as alive in that poem, and others in a similar vein. With Stephen Ramsay's notion of algorithmic criticism in mind, I wanted to turn Garcilaso's poems into unfamiliar objects in the hope that the process might help me better understand them on the other side of processing.

This project consists of a first round of text analysis, in which I find the most frequently used words in Garcilaso's poetic corpus. In the second part, I use Latent Dirichlet Allocation to plot out topics in Garcilaso's poems.

### Data and Works Cited

I use a collection of Garcilaso de la Vega's poems from Project Gutenberg. I've listed below the works that influenced my thinking around Garcilaso's poetry, but also about how to execute this project:
 - Middlebrook, Leah. _Imperial Lyric. Penn State University Press_, 2009.
 - Navarrete, Ignacio Enrique. _Orphans of Petrarch: Poetry and Theory in the Spanish Renaissance._ University of California Press, 1994.
 - Rhody, Lisa. “Topic Modeling and Figurative Language.” Journal of Digital Humanities, vol. 2, no. 1, Winter 2012, https://journalofdigitalhumanities.org/2-1/topic-modeling-and-figurative-language-by-lisa-m-rhody/.
 - Ramsay, Stephen. _Reading Machines: Toward an Algorithmic Criticism_. University of Illinois Press, 2011.

In [2]:
#!pip install pandas
import pandas as pd

In [3]:
import numpy as np
import re 
import nltk
from nltk.stem import WordNetLemmatizer as lemmatizer
from nltk import word_tokenize
import nltk.corpus
from nltk.corpus import stopwords
import string
from string import punctuation
import gensim
import spacy

In [ ]:
#!pip install spacy_spanish_lemmatizer

In [ ]:
#!python -m spacy_spanish_lemmatizer download wiki

In [ ]:
#!python -m spacy download es_core_news_lg

## Cleaning the text for a first round of analysis

I anticipated having a few challenges related to the age and language of the text; 16th-century Spanish is, of course, slightly different from the language used nowadays. Here are the main issues that came up as I cleaned the text:
 - The standard punctuation list did not include characters commonly used in Spanish: ¿ and ¡. I had to add both "by hand". 
 - Because we're dealing with poetry here, I had to figure out a way to remove line numbers.
 - The standard list of Spanish stop words did not include a few archaic expressions, such as the word "agora", which has transformed over the centuries into its current form: "ahora", meaning "now". It also missed a few key words and expressions that I would consider stop words in Spanish, such as "si" ("if") and "allá" ("there"). 

In [97]:
#Importing the large Spanish language model from NLTK
import es_core_news_lg
nlp = es_core_news_lg.load()

In [98]:
#Checking we're in the right place
!pwd

/Users/chloemauvais/Practicing the Digital Humanities (Summer 2023)


In [99]:
#Loading the text file
garcilaso = open("garcilaso.txt")

In [10]:
garcilaso = open("garcilaso.txt", encoding = "utf-8").read()

In [11]:
print(garcilaso)

﻿The Project Gutenberg eBook of Obras, by Garcilaso de la Vega

This eBook is for the use of anyone anywhere in the United States and
most other parts of the world at no cost and with almost no restrictions
whatsoever. You may copy it, give it away or re-use it under the terms
of the Project Gutenberg License included with this eBook or online at
www.gutenberg.org. If you are not located in the United States, you
will have to check the laws of the country where you are located before
using this eBook.

Title: Obras

Author: Garcilaso de la Vega

Editor: Tomás Navarro Tomás

Release Date: May 20, 2022 [eBook #68131]

Language: Spanish

Produced by: Ramón Pajares Box and the Online Distributed Proofreading
             Team at https://www.pgdp.net. (This file was produced from
             images generously made available by The Internet
             Archive/Canadian Libraries.)

*** START OF THE PROJECT GUTENBERG EBOOK OBRAS ***


NOTA DE TRANSCRIPCIÓN

  * Las cursivas se muestran entr

In [12]:
obras_completas = garcilaso

In [13]:
#Finding the beginning of the poems
garcilaso.rindex("ÉGLOGA PRIMERA")

19233

In [14]:
#Searching for the last line in the last poem
garcilaso.rindex("sino que os haya mirado.")

220557

In [15]:
#Indexing on the last character before the beginning of the apendices
print(garcilaso[220586:])

APÉNDICES

I

A LA MUY MAGNÍFICA SEÑORA DOÑA JERÓNIMA PALOVA DE ALMOGÁVAR, GARCILASO
DE LA VEGA[399]


Si no hubiera sabido antes de ahora dónde llega el juicio de V. m.
bastárame para entenderlo ver que os parescía bien este libro; mas
ya estábades tan adelante en mi opinión que paresciéndome este libro
bien hasta ahora por muchas causas, la principal por donde ahora me lo
paresce es porque le habéis aprobado de tal manera, que podemos decir
que le habéis hecho, pues por vuestra causa le alcanzamos a tener en
lengua que le entendemos. Porque, no solamente no pensé poder acabar
con Boscán que le tradujese,[400] mas nunca me osé poner en decírselo,
según le vía siempre aborrecer a los que romanzan libros, aunque él a
esto no lo llama romanzar, ni yo tampoco,[401] mas aunque lo fuera creo
que no se escusara dello mandándolo V. m.

Estoy muy satisfecho de mí, porque antes que el libro viniese a
vuestras manos,[402] ya yo le tenía en tanto como entonces debía;
porque si ahora, después que 

In [16]:
#Creating a trimmed version of the text file
#This version runs only from the title of the first poem, Égloga primera, to the last blank before the apendices
garcilaso_trimmed = garcilaso[19233:220586]
print(garcilaso_trimmed)

ÉGLOGA PRIMERA


      El dulce lamentar de dos pastores,[27]
    Salicio juntamente y Nemoroso,[28]
    he de contar, sus quejas imitando;
    cuyas ovejas al cantar sabroso
    estaban muy atentas, los amores,                                   5
    de pacer olvidadas, escuchando.
    Tú, que ganaste obrando
    un nombre en todo el mundo,
    y un grado sin segundo,
    agora estés atento, solo y dado                                   10
    al ínclito gobierno del Estado
    albano; agora vuelto a la otra parte,[29]
    resplandeciente, armado,
    representando en tierra el fiero Marte;[30]
      agora de cuidados enojosos                                      15
    y de negocios libre, por ventura
    andes a caza, el monte fatigando
    en ardiente jinete, que apresura
    el curso tras los ciervos temerosos,
    que en vano su morir van dilatando;                               20
    espera, que en tornando
    a ser restituído
    al ocio ya perdido,
    luego verás ejercitar 

In [17]:
len(garcilaso_trimmed)

201353

In [18]:
garcilaso_trimmed.lower()

'égloga primera\n\n\n      el dulce lamentar de dos pastores,[27]\n    salicio juntamente y nemoroso,[28]\n    he de contar, sus quejas imitando;\n    cuyas ovejas al cantar sabroso\n    estaban muy atentas, los amores,                                   5\n    de pacer olvidadas, escuchando.\n    tú, que ganaste obrando\n    un nombre en todo el mundo,\n    y un grado sin segundo,\n    agora estés atento, solo y dado                                   10\n    al ínclito gobierno del estado\n    albano; agora vuelto a la otra parte,[29]\n    resplandeciente, armado,\n    representando en tierra el fiero marte;[30]\n      agora de cuidados enojosos                                      15\n    y de negocios libre, por ventura\n    andes a caza, el monte fatigando\n    en ardiente jinete, que apresura\n    el curso tras los ciervos temerosos,\n    que en vano su morir van dilatando;                               20\n    espera, que en tornando\n    a ser restituído\n    al ocio ya perdido,\

#### Removing Punctuation

In [19]:
#for loop removing punctuation from my trimmed version of the poems
for char in punctuation: 
    garcilaso_trimmed = garcilaso_trimmed.replace(char,"")

In [20]:
print(garcilaso_trimmed)

ÉGLOGA PRIMERA


      El dulce lamentar de dos pastores27
    Salicio juntamente y Nemoroso28
    he de contar sus quejas imitando
    cuyas ovejas al cantar sabroso
    estaban muy atentas los amores                                   5
    de pacer olvidadas escuchando
    Tú que ganaste obrando
    un nombre en todo el mundo
    y un grado sin segundo
    agora estés atento solo y dado                                   10
    al ínclito gobierno del Estado
    albano agora vuelto a la otra parte29
    resplandeciente armado
    representando en tierra el fiero Marte30
      agora de cuidados enojosos                                      15
    y de negocios libre por ventura
    andes a caza el monte fatigando
    en ardiente jinete que apresura
    el curso tras los ciervos temerosos
    que en vano su morir van dilatando                               20
    espera que en tornando
    a ser restituído
    al ocio ya perdido
    luego verás ejercitar mi pluma
    por la infinita inn

If you scroll through the poems above, you'll notice the for loop failed to remove _all_ the punctuation. That's because Python's punctuation library does not include two common Spanish characters: ¿ and ¡

See the list of included punctuation below.

In [21]:
print(punctuation)

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~


I want to remove the exclamatory and interrogatory punctuation in the poems, as well as the line numbers, so I'll do both below. 

First, I'll remove the Spanish punctuation using a method I found [here](https://favtutor.com/blogs/replace-multiple-characters-in-string-python#:~:text=You%20can%20also%20replace%20multiple,function%20with%20lists%20and%20dictionaries).

In [22]:
spanish_punct = ["¿","¡"]
for char in spanish_punct:
    garcilaso_trimmed = garcilaso_trimmed.replace(char, "")
print(garcilaso_trimmed)

ÉGLOGA PRIMERA


      El dulce lamentar de dos pastores27
    Salicio juntamente y Nemoroso28
    he de contar sus quejas imitando
    cuyas ovejas al cantar sabroso
    estaban muy atentas los amores                                   5
    de pacer olvidadas escuchando
    Tú que ganaste obrando
    un nombre en todo el mundo
    y un grado sin segundo
    agora estés atento solo y dado                                   10
    al ínclito gobierno del Estado
    albano agora vuelto a la otra parte29
    resplandeciente armado
    representando en tierra el fiero Marte30
      agora de cuidados enojosos                                      15
    y de negocios libre por ventura
    andes a caza el monte fatigando
    en ardiente jinete que apresura
    el curso tras los ciervos temerosos
    que en vano su morir van dilatando                               20
    espera que en tornando
    a ser restituído
    al ocio ya perdido
    luego verás ejercitar mi pluma
    por la infinita inn

No more punctuation, but we're left with the line numbers. I don't want those either.

#### Removing Line Numbers

I use method number two from this web page, [\"Python | Ways to remove numeric digits from given string\"](https://www.geeksforgeeks.org/python-ways-to-remove-numeric-digits-from-given-string/#), which seems like the easiest way to clear out those numbers. It's also consistent with the method I've used so far, since it leans on the string library we're already employing to clean out punctuation.

In [23]:
#Removing line numbers from the poems
from string import digits 
remove_digits = str.maketrans("","", digits)
garcilaso_trimmed_nd = garcilaso_trimmed.translate(remove_digits)
print(garcilaso_trimmed_nd)

ÉGLOGA PRIMERA


      El dulce lamentar de dos pastores
    Salicio juntamente y Nemoroso
    he de contar sus quejas imitando
    cuyas ovejas al cantar sabroso
    estaban muy atentas los amores                                   
    de pacer olvidadas escuchando
    Tú que ganaste obrando
    un nombre en todo el mundo
    y un grado sin segundo
    agora estés atento solo y dado                                   
    al ínclito gobierno del Estado
    albano agora vuelto a la otra parte
    resplandeciente armado
    representando en tierra el fiero Marte
      agora de cuidados enojosos                                      
    y de negocios libre por ventura
    andes a caza el monte fatigando
    en ardiente jinete que apresura
    el curso tras los ciervos temerosos
    que en vano su morir van dilatando                               
    espera que en tornando
    a ser restituído
    al ocio ya perdido
    luego verás ejercitar mi pluma
    por la infinita innumerable suma  

In [24]:
garcilaso_trimmed_nd.split()

['ÉGLOGA',
 'PRIMERA',
 'El',
 'dulce',
 'lamentar',
 'de',
 'dos',
 'pastores',
 'Salicio',
 'juntamente',
 'y',
 'Nemoroso',
 'he',
 'de',
 'contar',
 'sus',
 'quejas',
 'imitando',
 'cuyas',
 'ovejas',
 'al',
 'cantar',
 'sabroso',
 'estaban',
 'muy',
 'atentas',
 'los',
 'amores',
 'de',
 'pacer',
 'olvidadas',
 'escuchando',
 'Tú',
 'que',
 'ganaste',
 'obrando',
 'un',
 'nombre',
 'en',
 'todo',
 'el',
 'mundo',
 'y',
 'un',
 'grado',
 'sin',
 'segundo',
 'agora',
 'estés',
 'atento',
 'solo',
 'y',
 'dado',
 'al',
 'ínclito',
 'gobierno',
 'del',
 'Estado',
 'albano',
 'agora',
 'vuelto',
 'a',
 'la',
 'otra',
 'parte',
 'resplandeciente',
 'armado',
 'representando',
 'en',
 'tierra',
 'el',
 'fiero',
 'Marte',
 'agora',
 'de',
 'cuidados',
 'enojosos',
 'y',
 'de',
 'negocios',
 'libre',
 'por',
 'ventura',
 'andes',
 'a',
 'caza',
 'el',
 'monte',
 'fatigando',
 'en',
 'ardiente',
 'jinete',
 'que',
 'apresura',
 'el',
 'curso',
 'tras',
 'los',
 'ciervos',
 'temerosos',
 'qu

In [25]:
garcilaso_words = garcilaso_trimmed_nd.lower().split()

In [26]:
#Looks like we've cleaned out the punctuation and numbers from our text.
print(garcilaso_words)

['égloga', 'primera', 'el', 'dulce', 'lamentar', 'de', 'dos', 'pastores', 'salicio', 'juntamente', 'y', 'nemoroso', 'he', 'de', 'contar', 'sus', 'quejas', 'imitando', 'cuyas', 'ovejas', 'al', 'cantar', 'sabroso', 'estaban', 'muy', 'atentas', 'los', 'amores', 'de', 'pacer', 'olvidadas', 'escuchando', 'tú', 'que', 'ganaste', 'obrando', 'un', 'nombre', 'en', 'todo', 'el', 'mundo', 'y', 'un', 'grado', 'sin', 'segundo', 'agora', 'estés', 'atento', 'solo', 'y', 'dado', 'al', 'ínclito', 'gobierno', 'del', 'estado', 'albano', 'agora', 'vuelto', 'a', 'la', 'otra', 'parte', 'resplandeciente', 'armado', 'representando', 'en', 'tierra', 'el', 'fiero', 'marte', 'agora', 'de', 'cuidados', 'enojosos', 'y', 'de', 'negocios', 'libre', 'por', 'ventura', 'andes', 'a', 'caza', 'el', 'monte', 'fatigando', 'en', 'ardiente', 'jinete', 'que', 'apresura', 'el', 'curso', 'tras', 'los', 'ciervos', 'temerosos', 'que', 'en', 'vano', 'su', 'morir', 'van', 'dilatando', 'espera', 'que', 'en', 'tornando', 'a', 'ser', 

#### Removing Stop Words

Now that I've cleaned out the punctuation and numbers from the text and converted it into a list of individual words, I want to remove stop words. If this text were in English, I'd use the provided NLTK stopwords list, but since I'm working on a set of Spanish poems from the 16th century, I might want to modify the list of stopwords in the NLTK library. 

Let's check out what those stopwords are.

In [27]:
nltk.corpus.stopwords.words('spanish')

['de',
 'la',
 'que',
 'el',
 'en',
 'y',
 'a',
 'los',
 'del',
 'se',
 'las',
 'por',
 'un',
 'para',
 'con',
 'no',
 'una',
 'su',
 'al',
 'lo',
 'como',
 'más',
 'pero',
 'sus',
 'le',
 'ya',
 'o',
 'este',
 'sí',
 'porque',
 'esta',
 'entre',
 'cuando',
 'muy',
 'sin',
 'sobre',
 'también',
 'me',
 'hasta',
 'hay',
 'donde',
 'quien',
 'desde',
 'todo',
 'nos',
 'durante',
 'todos',
 'uno',
 'les',
 'ni',
 'contra',
 'otros',
 'ese',
 'eso',
 'ante',
 'ellos',
 'e',
 'esto',
 'mí',
 'antes',
 'algunos',
 'qué',
 'unos',
 'yo',
 'otro',
 'otras',
 'otra',
 'él',
 'tanto',
 'esa',
 'estos',
 'mucho',
 'quienes',
 'nada',
 'muchos',
 'cual',
 'poco',
 'ella',
 'estar',
 'estas',
 'algunas',
 'algo',
 'nosotros',
 'mi',
 'mis',
 'tú',
 'te',
 'ti',
 'tu',
 'tus',
 'ellas',
 'nosotras',
 'vosotros',
 'vosotras',
 'os',
 'mío',
 'mía',
 'míos',
 'mías',
 'tuyo',
 'tuya',
 'tuyos',
 'tuyas',
 'suyo',
 'suya',
 'suyos',
 'suyas',
 'nuestro',
 'nuestra',
 'nuestros',
 'nuestras',
 'vuestro'

I notice this list is missing a few of the more archaic terms in Garcilaso's poetry, like "agora" (meaning the present-day word "ahora", or "now"). I'm going to try removing the stopwords with the list as is, and if I find other unwanted words floating to the top, I'll remove them later.

In [102]:
garcilaso_nostops = [word for word in garcilaso_words if word not in stopwords.words('spanish')]
print(garcilaso_nostops)

['égloga', 'primera', 'dulce', 'lamentar', 'dos', 'pastores', 'salicio', 'juntamente', 'nemoroso', 'contar', 'quejas', 'imitando', 'cuyas', 'ovejas', 'cantar', 'sabroso', 'atentas', 'amores', 'pacer', 'olvidadas', 'escuchando', 'ganaste', 'obrando', 'nombre', 'mundo', 'grado', 'segundo', 'agora', 'atento', 'solo', 'dado', 'ínclito', 'gobierno', 'albano', 'agora', 'vuelto', 'parte', 'resplandeciente', 'armado', 'representando', 'tierra', 'fiero', 'marte', 'agora', 'cuidados', 'enojosos', 'negocios', 'libre', 'ventura', 'andes', 'caza', 'monte', 'fatigando', 'ardiente', 'jinete', 'apresura', 'curso', 'tras', 'ciervos', 'temerosos', 'vano', 'morir', 'van', 'dilatando', 'espera', 'tornando', 'ser', 'restituído', 'ocio', 'perdido', 'luego', 'verás', 'ejercitar', 'pluma', 'infinita', 'innumerable', 'suma', 'virtudes', 'famosas', 'obras', 'consuma', 'faltando', 'mundo', 'sobras', 'tiempo', 'adivino', 'viene', 'sacarme', 'deuda', 'día', 'debe', 'fama', 'gloria', 'deuda', 'general', 'solo', 'ma

In [103]:
len(garcilaso_nostops)

14520

### Counting the most frequent words in the collection

In [29]:
#I want to count the most frequent words in the collection
from collections import Counter

In [70]:
garcilaso_freq = Counter(garcilaso_nostops)
garcilaso_freq.most_common(20)

[('si', 147),
 ('mas', 101),
 ('tan', 87),
 ('bien', 85),
 ('mal', 85),
 ('ser', 76),
 ('vida', 70),
 ('aquel', 70),
 ('tal', 67),
 ('así', 66),
 ('aquí', 65),
 ('parte', 64),
 ('allí', 64),
 ('solo', 60),
 ('oh', 57),
 ('gran', 57),
 ('albanio', 55),
 ('mano', 54),
 ('salicio', 52),
 ('do', 50)]

There are _a lot_ of stopwords here still!

I'm going to add a few words that are overrepresented in my data to the a new list of stopwords I'll be removing from the poems. Those include "aquí", meaning "here", "allí", meaning "there", "así", which means "thus", "si", which means "if", "tal", which means "such", and "tan", which can mean "so" or "such". I've also added "vos", which is an uncommonly used pronoun, meaning "you". Also, "aquella" (that) and "agora" (antiquated way of saying "ahora", now).

In [71]:
span_stopwords_new = stopwords.words('spanish') + ["aquí","así","allí","allá", "oh", "mas", "si","tal","tan","vos","aquella","aquel", "agora","do", "yo"]

In [100]:
print(span_stopwords_new)

['de', 'la', 'que', 'el', 'en', 'y', 'a', 'los', 'del', 'se', 'las', 'por', 'un', 'para', 'con', 'no', 'una', 'su', 'al', 'lo', 'como', 'más', 'pero', 'sus', 'le', 'ya', 'o', 'este', 'sí', 'porque', 'esta', 'entre', 'cuando', 'muy', 'sin', 'sobre', 'también', 'me', 'hasta', 'hay', 'donde', 'quien', 'desde', 'todo', 'nos', 'durante', 'todos', 'uno', 'les', 'ni', 'contra', 'otros', 'ese', 'eso', 'ante', 'ellos', 'e', 'esto', 'mí', 'antes', 'algunos', 'qué', 'unos', 'yo', 'otro', 'otras', 'otra', 'él', 'tanto', 'esa', 'estos', 'mucho', 'quienes', 'nada', 'muchos', 'cual', 'poco', 'ella', 'estar', 'estas', 'algunas', 'algo', 'nosotros', 'mi', 'mis', 'tú', 'te', 'ti', 'tu', 'tus', 'ellas', 'nosotras', 'vosotros', 'vosotras', 'os', 'mío', 'mía', 'míos', 'mías', 'tuyo', 'tuya', 'tuyos', 'tuyas', 'suyo', 'suya', 'suyos', 'suyas', 'nuestro', 'nuestra', 'nuestros', 'nuestras', 'vuestro', 'vuestra', 'vuestros', 'vuestras', 'esos', 'esas', 'estoy', 'estás', 'está', 'estamos', 'estáis', 'están', 'e

In [101]:
#Lets run that again
garcilaso_nostops2 = [word for word in garcilaso_words if word not in span_stopwords_new]
print(garcilaso_nostops2)

['égloga', 'primera', 'dulce', 'lamentar', 'dos', 'pastores', 'salicio', 'juntamente', 'nemoroso', 'contar', 'quejas', 'imitando', 'cuyas', 'ovejas', 'cantar', 'sabroso', 'atentas', 'amores', 'pacer', 'olvidadas', 'escuchando', 'ganaste', 'obrando', 'nombre', 'mundo', 'grado', 'segundo', 'atento', 'solo', 'dado', 'ínclito', 'gobierno', 'albano', 'vuelto', 'parte', 'resplandeciente', 'armado', 'representando', 'tierra', 'fiero', 'marte', 'cuidados', 'enojosos', 'negocios', 'libre', 'ventura', 'andes', 'caza', 'monte', 'fatigando', 'ardiente', 'jinete', 'apresura', 'curso', 'tras', 'ciervos', 'temerosos', 'vano', 'morir', 'van', 'dilatando', 'espera', 'tornando', 'ser', 'restituído', 'ocio', 'perdido', 'luego', 'verás', 'ejercitar', 'pluma', 'infinita', 'innumerable', 'suma', 'virtudes', 'famosas', 'obras', 'consuma', 'faltando', 'mundo', 'sobras', 'tiempo', 'adivino', 'viene', 'sacarme', 'deuda', 'día', 'debe', 'fama', 'gloria', 'deuda', 'general', 'solo', 'cualquier', 'ingenio', 'pereg

In [74]:
len(garcilaso_nostops2)

13591

This new version of the text is almost 1,000 words shorter than before I edited the list of stop words!

In [75]:
garcilaso_freq2 = Counter(garcilaso_nostops2)
garcilaso_freq2.most_common(30)

[('bien', 85),
 ('mal', 85),
 ('ser', 76),
 ('vida', 70),
 ('parte', 64),
 ('solo', 60),
 ('gran', 57),
 ('albanio', 55),
 ('mano', 54),
 ('salicio', 52),
 ('fin', 50),
 ('ojos', 47),
 ('dulce', 46),
 ('alma', 45),
 ('muerte', 45),
 ('tierra', 42),
 ('agua', 41),
 ('ver', 41),
 ('luego', 40),
 ('dolor', 40),
 ('pues', 39),
 ('amor', 39),
 ('quién', 39),
 ('cielo', 36),
 ('cuanto', 35),
 ('nunca', 35),
 ('fuerza', 35),
 ('siempre', 34),
 ('día', 33),
 ('cosa', 33)]

These results didn't feel all that insightful at first, but the more I look at this list, the more interesting I find it. I'm especially interested in how often Garcilaso discusses mal, bien and ser: bad, good, and being. Since I work on how people write about nature, the fact that Garcilaso mentions "tierra" (earth), "agua" (water), and "cielo" (sky) so often also piques my interest. 

My initial impression from this list is that Garcilaso's poems tend to trend toward the sad, with so many instances of words like "mal", "fin", "muerte", "dolor", "triste", etc. If it weren't for the huge amount of Garcilaso's poetry on offer, I think counting the frequency of words in his corpus would yield significantly less interesting results. Since there are over 14,000 words in his poems (not counting stop words), there is a lot to look through. 

### Using LDA to extract a list of topics from Garcilaso's poetry

In [76]:
#Here's the mostly unprocessed string from the beginning of this project. 
garcilaso_trimmed_nd

'ÉGLOGA PRIMERA\n\n\n      El dulce lamentar de dos pastores\n    Salicio juntamente y Nemoroso\n    he de contar sus quejas imitando\n    cuyas ovejas al cantar sabroso\n    estaban muy atentas los amores                                   \n    de pacer olvidadas escuchando\n    Tú que ganaste obrando\n    un nombre en todo el mundo\n    y un grado sin segundo\n    agora estés atento solo y dado                                   \n    al ínclito gobierno del Estado\n    albano agora vuelto a la otra parte\n    resplandeciente armado\n    representando en tierra el fiero Marte\n      agora de cuidados enojosos                                      \n    y de negocios libre por ventura\n    andes a caza el monte fatigando\n    en ardiente jinete que apresura\n    el curso tras los ciervos temerosos\n    que en vano su morir van dilatando                               \n    espera que en tornando\n    a ser restituído\n    al ocio ya perdido\n    luego verás ejercitar mi pluma\n    por la

In [77]:
#That text separated by lines
garci_lines = garcilaso_trimmed_nd.split("\n")

In [78]:
print(garci_lines)

['ÉGLOGA PRIMERA', '', '', '      El dulce lamentar de dos pastores', '    Salicio juntamente y Nemoroso', '    he de contar sus quejas imitando', '    cuyas ovejas al cantar sabroso', '    estaban muy atentas los amores                                   ', '    de pacer olvidadas escuchando', '    Tú que ganaste obrando', '    un nombre en todo el mundo', '    y un grado sin segundo', '    agora estés atento solo y dado                                   ', '    al ínclito gobierno del Estado', '    albano agora vuelto a la otra parte', '    resplandeciente armado', '    representando en tierra el fiero Marte', '      agora de cuidados enojosos                                      ', '    y de negocios libre por ventura', '    andes a caza el monte fatigando', '    en ardiente jinete que apresura', '    el curso tras los ciervos temerosos', '    que en vano su morir van dilatando                               ', '    espera que en tornando', '    a ser restituído', '    al ocio ya perd

I spent a long time trying to figure out how to split the text by poems, until someone pointed out that, in the original text file, everything was indented _except_ the titles of the poems. Below is my attempt to separate the poems using that feature of the text.

In [79]:
n_poems = 0
garci_dict = dict()
for line in garci_lines:
    if len(line) > 0: #line is not empty, since we only want text
        if line[0] != " ": #line is a title because it begins with a character rather than a space
            #it can only begin when we have one poem
            if n_poems > 0:
                garci_dict[title] = newpoem
            # our new poem starts with the title (hopefully!)
            newpoem = ''
            title = line
            # counting the next poem
            n_poems += 1
        #if the line does NOT start with text, meaning it is *indented*, then we have the body of a poem
        #and that needs to be added into the existing newpoem variable
        else:
            newpoem += line
        #add the line to the existing poem
#Stores the last poem
garci_dict[title] = newpoem

print(garci_dict.keys())
print(garci_dict['IX'])

dict_keys(['ÉGLOGA PRIMERA', 'ÉGLOGA SEGUNDA', 'ÉGLOGA III', 'ELEGÍA PRIMERA', 'ELEGÍA SEGUNDA', 'EPÍSTOLA', 'CANCIÓN PRIMERA', 'CANCIÓN SEGUNDA', 'CANCIÓN TERCERA', 'CANCIÓN CUARTA', 'CANCIÓN QUINTA', 'SONETOS', 'I', 'II', 'III', 'IV', 'V', 'VI', 'VII', 'VIII', 'IX', 'X', 'XI', 'XII', 'XIII', 'XIV', 'XV', 'XVI', 'XVII', 'XVIII', 'XIX', 'XX', 'XXI', 'XXII', 'XXIII', 'XXIV', 'XXV', 'XXVI', 'XXVII', 'XXVIII', 'XXIX', 'XXX', 'XXXI', 'XXXII', 'XXXIII', 'XXXIV', 'XXXV', 'XXXVI', 'XXXVII', 'XXXVIII', 'CANCIONES', 'EN VERSOS CORTOS'])
      Señora mía si de vos yo ausente    en esta vida turo y no me muero    paréceme que ofendo a lo que os quiero    y al bien de que gozaba en ser presente      Tras este luego siento otro acidente                               que es ver que si de vida desespero    yo pierdo cuanto bien de vos espero    y así ando en lo que siento diferente      En esta diferencia mis sentidos    están en vuestra ausencia y en porfía                                No sé ya qu

In [80]:
garci_dataframe = pd.DataFrame.from_dict(garci_dict, 
                                          orient = 'index', columns=['Poem_text']).reset_index()
garci_dataframe.rename(columns = {'index': 'Title'}) 

,Title,Poem_text
0,ÉGLOGA PRIMERA,El dulce lamentar de dos pastores Sal...
1,ÉGLOGA SEGUNDA,ALBANIO En medio del invierno está te...
2,ÉGLOGA III,Aquella voluntad honesta y pura ilust...
3,ELEGÍA PRIMERA,Aunque este grave caso haya tocado co...
4,ELEGÍA SEGUNDA,Aquí Boscán donde del buen troyano An...
5,EPÍSTOLA,Señor Boscán quien tanto gusto tiene ...
6,CANCIÓN PRIMERA,Si a la región desierta inhabitable p...
7,CANCIÓN SEGUNDA,La soledad siguiendo rendido a mi for...
8,CANCIÓN TERCERA,Con un manso ruído de agua corriente ...
9,CANCIÓN CUARTA,El aspereza de mis males quiero que s...


Since our course covers only processing English texts, I had to look elsewhere for resources on doing the same with Spanish texts. My two main sources of information were ["Introducción al topic modeling con Gensim (I): Fundamentos y preprocesamiento de textos](https://elmundodelosdatos.com/topic-modeling-gensim-fundamentos-preprocesamiento-textos/), and ["Topic Modeling on Spanish Texts"](https://medium.com/@armandj.olivares/topic-modeling-on-spanish-text-f7a5e998fb90). 

In [81]:
#preprocessing the text as in notebook 5-2 & a few blog posts I found on processing spanish text
def clean_poems(poems): 
    poems = re.sub(r'\W', ' ', str(poems))
   #poems=re.sub(r'\s+[a-zA-Z]\s+', ' ', poems)
    poems = re.sub(r'\s+', ' ', poems, flags =re.I)
    poems = poems.lower()
    return poems
garci_dataframe['Poem_text_processed']= garci_dataframe['Poem_text'].apply(clean_poems)
garci_dataframe

,index,Poem_text,Poem_text_processed
0,ÉGLOGA PRIMERA,El dulce lamentar de dos pastores Sal...,el dulce lamentar de dos pastores salicio jun...
1,ÉGLOGA SEGUNDA,ALBANIO En medio del invierno está te...,albanio en medio del invierno está templada e...
2,ÉGLOGA III,Aquella voluntad honesta y pura ilust...,aquella voluntad honesta y pura ilustre y her...
3,ELEGÍA PRIMERA,Aunque este grave caso haya tocado co...,aunque este grave caso haya tocado con tanto ...
4,ELEGÍA SEGUNDA,Aquí Boscán donde del buen troyano An...,aquí boscán donde del buen troyano anquises c...
5,EPÍSTOLA,Señor Boscán quien tanto gusto tiene ...,señor boscán quien tanto gusto tiene de daros...
6,CANCIÓN PRIMERA,Si a la región desierta inhabitable p...,si a la región desierta inhabitable por el he...
7,CANCIÓN SEGUNDA,La soledad siguiendo rendido a mi for...,la soledad siguiendo rendido a mi fortuna me ...
8,CANCIÓN TERCERA,Con un manso ruído de agua corriente ...,con un manso ruído de agua corriente y clara ...
9,CANCIÓN CUARTA,El aspereza de mis males quiero que s...,el aspereza de mis males quiero que se muestr...


In [82]:
#Checking how one of the poems looks
garci_dataframe.loc[46].at['Poem_text_processed']

' boscán las armas y el furor de marte que con su propia sangre el africano suelo regando hacen que el romano imperio reverdesca en esta parte han reducido a la memoria el arte y el antiguo valor italiano por cuya fuerza y valerosa mano áfrica se aterró de parte a parte aquí donde el romano encendimiento donde el fuego y la llama licenciosa solo el nombre dejaron a cartago vuelve y revuelve amor mi pensamiento hiere y enciende el alma temerosa y en llanto y en ceniza me deshago'

In the blog posts I cited above, both writers chose to stem the words in their texts. I prefer to use lemmatization since it "uses lexical knowledge bases to get the correct base forms of words." [See this blog post for more information](https://medium.com/mlearning-ai/nlp-tokenization-stemming-lemmatization-and-part-of-speech-tagging-9088ac068768). I'm going to be using the recently released spaCy Spanish lemmatizer.

In [83]:
from nltk.tokenize import ToktokTokenizer
tokenizer = ToktokTokenizer()

In [84]:
#first I'm gonna tokenize the text
garci_dataframe['Tokens'] = garci_dataframe.Poem_text_processed.apply(tokenizer.tokenize)

In [85]:
garci_dataframe

,index,Poem_text,Poem_text_processed,Tokens
0,ÉGLOGA PRIMERA,El dulce lamentar de dos pastores Sal...,el dulce lamentar de dos pastores salicio jun...,"[el, dulce, lamentar, de, dos, pastores, salic..."
1,ÉGLOGA SEGUNDA,ALBANIO En medio del invierno está te...,albanio en medio del invierno está templada e...,"[albanio, en, medio, del, invierno, está, temp..."
2,ÉGLOGA III,Aquella voluntad honesta y pura ilust...,aquella voluntad honesta y pura ilustre y her...,"[aquella, voluntad, honesta, y, pura, ilustre,..."
3,ELEGÍA PRIMERA,Aunque este grave caso haya tocado co...,aunque este grave caso haya tocado con tanto ...,"[aunque, este, grave, caso, haya, tocado, con,..."
4,ELEGÍA SEGUNDA,Aquí Boscán donde del buen troyano An...,aquí boscán donde del buen troyano anquises c...,"[aquí, boscán, donde, del, buen, troyano, anqu..."
5,EPÍSTOLA,Señor Boscán quien tanto gusto tiene ...,señor boscán quien tanto gusto tiene de daros...,"[señor, boscán, quien, tanto, gusto, tiene, de..."
6,CANCIÓN PRIMERA,Si a la región desierta inhabitable p...,si a la región desierta inhabitable por el he...,"[si, a, la, región, desierta, inhabitable, por..."
7,CANCIÓN SEGUNDA,La soledad siguiendo rendido a mi for...,la soledad siguiendo rendido a mi fortuna me ...,"[la, soledad, siguiendo, rendido, a, mi, fortu..."
8,CANCIÓN TERCERA,Con un manso ruído de agua corriente ...,con un manso ruído de agua corriente y clara ...,"[con, un, manso, ruído, de, agua, corriente, y..."
9,CANCIÓN CUARTA,El aspereza de mis males quiero que s...,el aspereza de mis males quiero que se muestr...,"[el, aspereza, de, mis, males, quiero, que, se..."


In [104]:
#Defining a function to remove stopwords (using my new list!)
def nostopwords(tokens):
    return [token for token in tokens if token not in span_stopwords_new]
garci_dataframe['Tokens'] = garci_dataframe.Tokens.apply(nostopwords)
garci_dataframe

,index,Poem_text,Poem_text_processed,Tokens,Lemmas
0,ÉGLOGA PRIMERA,El dulce lamentar de dos pastores Sal...,el dulce lamentar de dos pastores salicio jun...,"[dulce, lamentar, dos, pastores, salicio, junt...",dulce lamentar dos pastor salicio juntamente n...
1,ÉGLOGA SEGUNDA,ALBANIO En medio del invierno está te...,albanio en medio del invierno está templada e...,"[albanio, medio, invierno, templada, agua, dul...",albanio medio invierno templado agua dulce des...
2,ÉGLOGA III,Aquella voluntad honesta y pura ilust...,aquella voluntad honesta y pura ilustre y her...,"[voluntad, honesta, pura, ilustre, hermosísima...",voluntad honesto puro ilustre hermosísima marí...
3,ELEGÍA PRIMERA,Aunque este grave caso haya tocado co...,aunque este grave caso haya tocado con tanto ...,"[aunque, grave, caso, tocado, sentimiento, alm...",aunque grave caso tocado sentimiento alma cons...
4,ELEGÍA SEGUNDA,Aquí Boscán donde del buen troyano An...,aquí boscán donde del buen troyano anquises c...,"[boscán, buen, troyano, anquises, eterno, nomb...",boscán buen troyano anquís eterno nombre vida ...
5,EPÍSTOLA,Señor Boscán quien tanto gusto tiene ...,señor boscán quien tanto gusto tiene de daros...,"[señor, boscán, gusto, daros, cuenta, pensamie...",señor boscán gusto dar tú cuenta pensamiento c...
6,CANCIÓN PRIMERA,Si a la región desierta inhabitable p...,si a la región desierta inhabitable por el he...,"[región, desierta, inhabitable, hervor, sol, d...",región desierto inhabitable hervor sol demasia...
7,CANCIÓN SEGUNDA,La soledad siguiendo rendido a mi for...,la soledad siguiendo rendido a mi fortuna me ...,"[soledad, siguiendo, rendido, fortuna, voy, ca...",soledad seguir rendir fortuna ir camino ofrece...
8,CANCIÓN TERCERA,Con un manso ruído de agua corriente ...,con un manso ruído de agua corriente y clara ...,"[manso, ruído, agua, corriente, clara, cerca, ...",manso ruído agua corriente claro cerca danubio...
9,CANCIÓN CUARTA,El aspereza de mis males quiero que s...,el aspereza de mis males quiero que se muestr...,"[aspereza, males, quiero, muestre, razones, ef...",aspereza mal querer mostrar razón efeto mostra...


In [105]:
#Checking what a poem looks like after removing the stopwords
garci_dataframe.loc[22].at['Tokens']

['hermosas',
 'ninfas',
 'río',
 'metidas',
 'contentas',
 'habitáis',
 'moradas',
 'relucientes',
 'piedras',
 'fabricadas',
 'colunas',
 'vidro',
 'sostenidas',
 'labrando',
 'embebecidas',
 'tejiendo',
 'telas',
 'delicadas',
 'unas',
 'apartadas',
 'contándoos',
 'amores',
 'vidas',
 'dejad',
 'rato',
 'labor',
 'alzando',
 'rubias',
 'cabezas',
 'mirarme',
 'detendréis',
 'según',
 'ando',
 'podréis',
 'lástima',
 'escucharme',
 'convertido',
 'agua',
 'llorando',
 'podréis',
 'espacio',
 'consolarme']

In [106]:
tokens = garci_dataframe['Tokens']

In [107]:
#Converting the list of tokens into a string 
garci_dataframe['Lemmas'] = [' '.join(t) for t in tokens]

In [90]:
garci_dataframe['Lemmas'][12][:2000]

'culpa debe ser quereros según hacéis pagaréis sabrán conoceros mal conocéis quereros ser perdido pensaba culpado sido mostrado bien sabido quién pudiese quereros sabéis holgarme paguéis conoceros conocéis'

In [91]:
nlp = spacy.load("es_core_news_lg", disable = ["ner", "textcat"])
print(type(nlp))

<class 'spacy.lang.es.Spanish'>


In [108]:
def lemmatize(text):
    doc = nlp(text)
    return ' '.join([word.lemma_ for word in doc])
garci_dataframe['Lemmas'] = garci_dataframe['Lemmas'].apply(lambda x: lemmatize(x))

In [111]:
#Printing the same poem, first in its tokenized form, then its lemmatized form
print(garci_dataframe['Tokens'][12][:2000])
print(garci_dataframe['Lemmas'][12][:2000])

['culpa', 'debe', 'ser', 'quereros', 'según', 'hacéis', 'pagaréis', 'sabrán', 'conoceros', 'mal', 'conocéis', 'quereros', 'ser', 'perdido', 'pensaba', 'culpado', 'sido', 'mostrado', 'bien', 'sabido', 'quién', 'pudiese', 'quereros', 'sabéis', 'holgarme', 'paguéis', 'conoceros', 'conocéis']
culpa deber ser querero según hacer pagaréi saber conocero mal conocéis querero ser perder pensar culpado ser mostrar bien saber quién poder querero sabéis holgarme pagueis  conocero conocéis


I have reservations about the usefulness of spaCy's Spanish-language package. It doesn't seem to have changed much about my text. Given more time and resources, I'd like to make modifications to the package. It's extremely inconsistent. It recognizes "hacéis" as the second-person plural of the verb "hacer", meaning "to do", but it does not do the same with the same tense and person in the word "pagaréis", of the verb "pagar", meaning "to pay". That inconsistency will absolutely warp my results, but I am unfortunately unsure of how to fix it without manually editing the words the package has missed.

### Creating the DTM
I want to input these texts to an LDA model, so first, I need to create a doctument term matrix.

In [94]:
#first, load vectorizer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, TfidfTransformer
vectorizer = CountVectorizer()
vectorizer.fit(garci_dataframe['Lemmas'])

CountVectorizer()

In [55]:
vector = vectorizer.transform(garci_dataframe['Lemmas'])

print(type(vector))
print(vector.shape)
print(vector)

<class 'scipy.sparse._csr.csr_matrix'>
(52, 3385)
  (0, 11)	1
  (0, 21)	1
  (0, 24)	1
  (0, 30)	1
  (0, 32)	1
  (0, 33)	2
  (0, 38)	2
  (0, 55)	1
  (0, 63)	1
  (0, 66)	1
  (0, 68)	1
  (0, 71)	1
  (0, 72)	1
  (0, 85)	2
  (0, 90)	1
  (0, 98)	1
  (0, 99)	1
  (0, 102)	7
  (0, 103)	1
  (0, 109)	1
  (0, 112)	2
  (0, 113)	2
  (0, 114)	1
  (0, 118)	1
  (0, 125)	1
  :	:
  (49, 1613)	1
  (49, 1846)	1
  (49, 1980)	1
  (49, 1981)	2
  (49, 2013)	1
  (49, 2077)	1
  (49, 2199)	1
  (49, 2331)	1
  (49, 2354)	1
  (49, 2409)	1
  (49, 2515)	1
  (49, 2580)	1
  (49, 2891)	3
  (49, 2899)	1
  (49, 2904)	1
  (49, 2906)	2
  (49, 2909)	2
  (49, 2914)	3
  (49, 2919)	2
  (49, 3024)	1
  (49, 3117)	1
  (49, 3206)	1
  (49, 3274)	1
  (49, 3361)	1
  (49, 3382)	1


In [56]:
print(vector.toarray())

[[0 0 0 ... 0 0 0]
 [1 2 0 ... 0 1 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 1 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [57]:
vectorizer.get_feature_names_out()[0:10]

array(['abajo', 'abandonar', 'abastanzo', 'abeja', 'abiertamente',
       'abierto', 'abismo', 'ablandacer', 'abollar', 'aborrecer'],
      dtype=object)

### Fitting the topic model

In [58]:
from sklearn.decomposition import LatentDirichletAllocation

In [59]:
n_topics = 10

In [112]:
tf_vectorizer = CountVectorizer(max_df = 0.95, max_features = 500)
cv = tf_vectorizer.fit_transform(garci_dataframe['Lemmas'])

In [113]:
lda = LatentDirichletAllocation(n_components = n_topics, 
                                max_iter = 20,
                                random_state = 42)
lda = lda.fit(cv)

In [114]:
def p_topwords(model, feature_names, n_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print("\nTopic #{}:".format(topic_idx+1))
        print(", ".join([feature_names[i] for i in topic.argsort()[:-n_top_words - 1:-1]]))

In [115]:
tf_feature_names = tf_vectorizer.get_feature_names_out()
p_topwords(lda, tf_feature_names, 20)


Topic #1:
ver, hablar, yo, dejar, hacer, mal, ser, día, nunca, aprovechar, locura, lágrima, pensar, sentir, acaso, fuego, escura, cada, esperanza, partir

Topic #2:
ver, ser, poder, albanio, dar, salicio, gran, hacer, yo, mal, bien, claro, mano, él, querer, vida, mostrar, solo, ir, decir

Topic #3:
blanco, verde, ir, agua, mostrar, ser, monte, ver, poder, hacer, oro, tajo, dar, todo, claro, labor, tirreno, alcino, vida, menos

Topic #4:
camino, cosa, hacer, gusto, ir, amistad, bien, dulce, llevar, nombre, gran, solo, pensamiento, amor, parte, fuerza, provecho, hallar, puro, pasado

Topic #5:
duro, miserable, ser, tornar, amante, llorar, áspero, hacer, crecer, deber, árbol, aspereza, sangre, poder, triste, fiero, cuello, convertir, tierra, airado

Topic #6:
mal, bien, poder, ir, saber, morir, solo, parte, hacer, yo, ver, ser, vida, temor, hora, alguno, quién, razón, mismo, pensar

Topic #7:
hacer, ver, bien, mal, vida, sentir, vía, sino, acidente, ausente, ausencia, perder, ser, saber,

These topics recall my point earlier about the difficulties of processing Spanish texts. I believe the lack of consistency in spaCy's lemmatization package trickled down into these results; many of the topics seem almost indistinguishable from each other. That either means there is striking consistency throughout Garcilaso's poems, or that there is a poverty of vocabulary for the model to parse. The topics are not all the same, however. I recognize in Topic #2 the names of the two shephers featured in one of Garcilaso's "Églogas", and the emphasis on seeing, being, and power intrigues me in its juxtaposition with the pastoral setting of the poem. There are also a few topics that seem to revolve around the idea of heartbreak, like #5, which is a list of bitterly sad words. That topic points to a throughline of Garcilaso's poetry: unresolved, unfulfilled romance. Topic #6, with its manichean emphasis on good/bad makes up another interesting list. These topics seem, for the most part, to form a road map for further exploration of Garcilaso's poetry. Do they make a net positive addition to my scholarly tools? I think so, and I think having this list of topics can enhance my reading in informative ways.

### Visualizing the topics

In [116]:
#pip install --upgrade pyldavis

In [117]:
#!pip install -U scikit-learn

In [118]:
import pyLDAvis
import sklearn
from pyLDAvis import lda_model
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, TfidfTransformer

In [119]:
#Class notebooks used the command "pyLDAvis.sklearn.prepare", but I couldn't get it to work
#I dug through pyLDAvis documentation and learned the developers had replaced the .sklearn command with what I've used below
panel = pyLDAvis.lda_model.prepare(lda_model = lda,
                                dtm = cv,
                                vectorizer = tf_vectorizer,
                                mds = "pcoa")
pyLDAvis.display(panel)

### Discussion

I can draw two parallel sets of conclusions from this project. 

#### Reflections on Using Computational Tools in this Project
First, my impressions on doing computational text analysis on old Spanish-language texts; I'm surprised by the lack of robust options for processing texts, especially with regards to lemmatization. I had to cobble together an approach from packages clearly meant first and foremost for English-language work, which may have interfered with my final results. I won't linger on this complaint any longer, since I've discussed it at length throughout this project notebook.

#### Project Results
The project still yielded some insights. The visualization I produced using the LDA results hints at the author's obsession with gazes ("ver") and power ("poder"), which match Middlebrook's arguments quite neatly. The topics also hint at some themes I hadn't previously considered in Garcilaso's writing, including a(n) (un)healthy dose of lovesickness. As I implied above, these topics form a useful road map for reading Garcilaso's poetry. With a deeper literature review, they could likely contribute to a greater understanding of his work. 